<a href="https://www.kaggle.com/code/thasankakandage/ufc-fight-predictor?scriptVersionId=193257294" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/d/thasankakandage/ufc-dataset-2024/ufc_fighters_avg.csv
/kaggle/input/d/thasankakandage/ufc-dataset-2024/ufc_event_fight_stats.csv
/kaggle/input/d/thasankakandage/ufc-dataset-2024/ufc_fighters_median.csv
/kaggle/input/d/thasankakandage/ufc-dataset-2024/ufc_fighters.csv
/kaggle/input/d/thasankakandage/ufc-dataset-2024/ufc_events.csv


# Event Fights Data Cleaning and Preprocessing

## Converting 'result' Section of Data


In the original, 'result' indicates the winner of the fight with either the ID of fighter1 (if fighter1 won), the ID of fighter2 (if fighter2 won), denotes a draw ('D' or 'd'),  or a ('nc') for a fight that had no contest.

In [2]:
train_data = pd.read_csv('/kaggle/input/d/thasankakandage/ufc-dataset-2024/ufc_event_fight_stats.csv') 

def map_result(row):

    if row['result'] == str(row['f1_id']):
        return 0
    elif row['result'] == str(row['f2_id']):
        return 1
    else:
      
        return row['result']

train_data['result'] = train_data.apply(map_result, axis=1)

print(train_data.info())
print(train_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7715 entries, 0 to 7714
Data columns (total 58 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   f1_id                7715 non-null   int64  
 1   f2_id                7715 non-null   int64  
 2   f1_name              7715 non-null   object 
 3   f2_name              7715 non-null   object 
 4   weight_class         7715 non-null   object 
 5   f1_age_during        7656 non-null   float64
 6   f2_age_during        7564 non-null   float64
 7   f1_height_cm         7708 non-null   float64
 8   f2_height_cm         7703 non-null   float64
 9   f1_knockdowns        7715 non-null   int64  
 10  f2_knockdowns        7715 non-null   int64  
 11  f1_sig_strike_atts   7715 non-null   int64  
 12  f2_sig_strike_atts   7715 non-null   int64  
 13  f1_sig_strikes       7715 non-null   int64  
 14  f2_sig_strikes       7715 non-null   int64  
 15  f1_tot_strike_atts   7715 non-null   i

## Converting 'weight_class' Section of Data from Categorical to Numerical

Remove rows where the weight_class is an outdated or irrelevant category, such as 'Ultimate '96 Tournament Title Bout'. This category represents an old UFC match style with unreliable statistics, many of which are not even displayed. Additionally, Open Weight and Catch Weight fights ARE counted, even if they are rare (~60 each). 

Note that women fights are not included in this dataset, due to two reasons:
1. Only about 10% of the entirety of fights that happened in the UFC are women fights.
2. Women fights and averages for them can be drastically different than mens.

In [3]:
print("Number of women fights:", train_data[train_data['weight_class'].str.contains("Women's", na=False)].shape[0])


def map_weight_class(row):
    if "Women's" in row['weight_class'] or "Strawweight" in row['weight_class']:
        return None
    elif "Flyweight" in row['weight_class']:
        return 0
    elif "Bantamweight" in row['weight_class']:
        return 1
    elif "Featherweight" in row['weight_class']:
        return 2
    elif "Flyweight" in row['weight_class']:
        return 3
    elif "Lightweight" in row['weight_class']:
        return 4
    elif "Welterweight" in row['weight_class']:
        return 5
    elif "Middleweight" in row['weight_class']:
        return 6
    elif "Light Heavyweight" in row['weight_class']:
        return 7
    elif "Heavyweight" in row['weight_class']:
        return 8
    elif "Catch Weight" in row['weight_class']:
        return 9
    elif "Open Weight" in row['weight_class']:
        return 10
    else:
        print(row['weight_class'], row['fights_url'])
        return None
   


train_data['weight_class'] = train_data.apply(map_weight_class, axis=1)
# Remove rows where 'weight_class' is something different, these ufc fights are very old anyways
train_data = train_data[train_data['weight_class'].notna()]

print(train_data.head())

Number of women fights: 777
Ultimate Ultimate '96 Tournament Title Bout http://www.ufcstats.com/fight-details/d595d2c36ddba8ee
UFC 10 Tournament Title Bout http://www.ufcstats.com/fight-details/6397fba14ce7f674
UFC Superfight Championship Bout http://www.ufcstats.com/fight-details/6a060498e60756af
UFC 8 Tournament Title Bout http://www.ufcstats.com/fight-details/8d2e99599124a16f
UFC Superfight Championship Bout http://www.ufcstats.com/fight-details/16b4a0b06427f1ac
Ultimate Ultimate '95 Tournament Title Bout http://www.ufcstats.com/fight-details/524b49a676498c6d
UFC Superfight Championship Bout http://www.ufcstats.com/fight-details/3932f8e9a74f3d11
UFC Superfight Championship Bout http://www.ufcstats.com/fight-details/db8df615610f3632
UFC 5 Tournament Title Bout http://www.ufcstats.com/fight-details/6ca94b35719eb300
UFC 3 Tournament Title Bout http://www.ufcstats.com/fight-details/323f543eb8abdb36
UFC 2 Tournament Title Bout http://www.ufcstats.com/fight-details/00835554f95fa911
   f1_

## Balancing the Dataset

Deleting rows where the result is not 0 or 1. This means that the fight ended up in a draw, or no contest which are extremely rare cases.

In [4]:
print(f"Number of rows in training data: {train_data.shape[0]}")
print(f'Number of rows where result is not 0 or 1: {train_data[~train_data["result"].isin([0, 1])].shape[0]}')
train_data = train_data[train_data['result'].isin([0, 1])]
print(f"Number of rows in training data after deletion: {train_data.shape[0]}")

# changing data type of result column from object to int
# before deleting the rows with result as neither 0 or 1, result can be "d" or "nc" which explains why the data type is "object" before changing
train_data['result'] = train_data['result'].astype('int64')





Number of rows in training data: 6927
Number of rows where result is not 0 or 1: 134
Number of rows in training data after deletion: 6793


The dataset has more rows where f1 won (result = 0) than f2. To balance the dataset, taking 1000 random rows with result = 0 and swapping the f1 and f2 stats and changing result from 0 to 1. 

In [5]:
# instead of choosing a static value like 1000, implement a system that checks the difference between classes and chooses n

print(f'Number of rows: {train_data.shape[0]}')
print(f'Number of rows with result 0 (f1 won): {train_data[train_data["result"] == 0].shape[0]}')
print(f'Number of rows with result 1 (f2 won): {train_data[train_data["result"] == 1].shape[0]}')

rows_to_modify = train_data[train_data['result'] == 0].sample(n=1150, random_state=42).index
columns_to_swap = [col for col in train_data.columns if col.startswith('f1_') or col.startswith('f2_')]


f1_columns = [col for col in columns_to_swap if col.startswith('f1_')]
f2_columns = [col for col in columns_to_swap if col.startswith('f2_')]


assert len(f1_columns) == len(f2_columns), "Mismatch between f1 and f2 columns"

show_sample = False
for idx in rows_to_modify:
    if not show_sample:
        print(train_data.loc[idx])
    
    for f1_col, f2_col in zip(f1_columns, f2_columns):
        
        train_data.at[idx, f1_col], train_data.at[idx, f2_col] = train_data.at[idx, f2_col], train_data.at[idx, f1_col]
    
    train_data.at[idx, 'result'] = 1
    if not show_sample:
        print(train_data.loc[idx])
        show_sample = True
    

print(f'Number of rows with result 0 (f1 won) after modification: {train_data[train_data["result"] == 0].shape[0]}')
print(f'Number of rows with result 1 (f2 won) after modification: {train_data[train_data["result"] == 1].shape[0]}')

Number of rows: 6793
Number of rows with result 0 (f1 won): 4463
Number of rows with result 1 (f2 won): 2330
f1_id                                                               4097
f2_id                                                                297
f1_name                                                   Khaos Williams
f2_name                                                   Rolando Bedoya
weight_class                                                         5.0
f1_age_during                                                       29.0
f2_age_during                                                       26.0
f1_height_cm                                                      182.88
f2_height_cm                                                      180.34
f1_knockdowns                                                          0
f2_knockdowns                                                          0
f1_sig_strike_atts                                                   283
f2_sig_strike_a

<a id="section3"></a>
### Cleaning Up Null Values
Cleaning up common null values inside the training data, like control time, birth date, height, etc. Since there are little amounts of data with the fighters age or height not stated (~200), I have decided to just omit the data. For control time, the ufc stats page sometimes has "--" which I believe means that this control time was either not tracked or was 0

In [6]:


null_counts_per_column = train_data.isnull().sum()

print("Null values per column:")
print(null_counts_per_column)

# these null values for ctrl time are usually "--" on a ufc stats page

train_data['f1_ctrl_time'] = train_data['f1_ctrl_time'].fillna(0)
train_data['f2_ctrl_time'] = train_data['f2_ctrl_time'].fillna(0)
train_data = train_data.dropna(subset=['f1_age_during', 'f2_age_during', 'f1_height_cm', 'f2_height_cm'])

print("Null values per column after replacement:")
print(train_data.isnull().sum())

print(f"Number of duplicate rows: {train_data.duplicated().sum()}")


Null values per column:
f1_id                    0
f2_id                    0
f1_name                  0
f2_name                  0
weight_class             0
f1_age_during           80
f2_age_during          123
f1_height_cm             8
f2_height_cm            11
f1_knockdowns            0
f2_knockdowns            0
f1_sig_strike_atts       0
f2_sig_strike_atts       0
f1_sig_strikes           0
f2_sig_strikes           0
f1_tot_strike_atts       0
f2_tot_strike_atts       0
f1_tot_strikes           0
f2_tot_strikes           0
f1_takedown_atts         0
f2_takedown_atts         0
f1_takedowns             0
f2_takedowns             0
f1_submissions           0
f2_submissions           0
f1_reversals             0
f2_reversals             0
f1_ctrl_time           136
f2_ctrl_time           136
f1_head_strike_atts      0
f2_head_strike_atts      0
f1_head_strikes          0
f2_head_strikes          0
f1_body_strike_atts      0
f2_body_strike_atts      0
f1_body_strikes          0
f2_b

# Fighter Data Cleaning and Preprocessing

## Median over Average Data
UFC fights are very extreme and vary widely in their outcomes, some fights might end in a matter of seconds which skews the average significantly, so I will choose to use the median over the average.

In [7]:



median_fighter_data = pd.read_csv('/kaggle/input/d/thasankakandage/ufc-dataset-2024/ufc_fighters_median.csv')
avg_fighter_data = pd.read_csv('/kaggle/input/d/thasankakandage/ufc-dataset-2024/ufc_fighters_avg.csv')



combined_data = pd.merge(avg_fighter_data, median_fighter_data, on='fighter_id', suffixes=('_avg', '_med'))


sampled_data = combined_data.sample(n=5)



comparison_columns = {
    'avg_knockdowns': 'Avg Knockdowns',
    'median_knockdowns': 'Median Knockdowns',
    'avg_sig_strike_atts': 'Avg Significant Strike Attempts',
    'median_sig_strike_atts': 'Median Significant Strike Attempts',
    'avg_sig_strikes': 'Avg Significant Strikes',
    'median_sig_strikes': 'Median Siginificant Strikes',
    'avg_tot_strike_atts': 'Avg Total Strike Attempts',
    'median_tot_strike_atts': 'Median Total Strike Attempts',
    'avg_tot_strikes': 'Avg Total Strikes',
    'median_tot_strikes': 'Median Total Strikes',
    'avg_takedown_atts': 'Avg Takedown Attempts',
    'median_takedown_atts': 'Median Takedown Attempts',
    'avg_takedowns': 'Avg Takedowns',
    'median_takedowns': 'Median Takedowns',
    'avg_clinch_atts': 'Avg Clinch Attempts',
    'median_clinch_atts': 'Median Clinch Attempts',
    'avg_clinchs': 'Avg Clinchs',
    'median_clinchs': 'Median Clinchs',
    'avg_ctrl_time': 'Avg Control Time',
    'median_ctrl_time': 'Median Control Time',
    'avg_total_fight_time': 'Avg Total Fight Time',
    'median_total_fight_time': 'Median Total Fight Time',
    'avg_submissions': 'Avg Submissions',
    'median_submissions': 'Median Submissions',
    'avg_reversals': 'Avg Reversals',
    'median_reversals': 'Median Reversals',
    'avg_head_strike_atts': 'Avg Head Strike Attempts',
    'median_head_strike_atts': 'Median Head Strike Attempts',
    'avg_head_strikes': 'Avg Head Strikes',
    'median_head_strikes': 'Median Head Strikes',
    'avg_body_strike_atts': 'Avg Body Strike Attempts',
    'median_body_strike_atts': 'Median Body Strike Attempts',
    'avg_body_strikes': 'Avg Body Strikes',
    'median_body_strikes': 'Median Body Strikes', 
    'avg_leg_strike_atts': 'Avg Leg Strike Attempts', 
    'median_leg_strike_atts': 'Median Leg Strike Attempts',
    'avg_leg_strikes': 'Avg Leg Strikes',
    'median_leg_strikes': 'Median Leg Strikes',
    'avg_dist_strike_atts': 'Avg Distance Strike Attempts',
    'median_dist_strike_atts': 'Median Distance Strike Attempts',
    'avg_dist_strikes': 'Average Distance Strikes',
    'median_dist_strikes': 'Median Distance Strikes',
    'avg_ground_atts': 'Average Ground Attempts',
    'median_ground_atts': 'Median Ground Attempts',
    'avg_grounds': 'Average Grounds',
    'median_grounds': 'Median Grounds'
}


existing_columns = [col for col in comparison_columns.keys() if col in sampled_data.columns]
comparison_data = sampled_data[existing_columns].rename(columns=comparison_columns)


print(comparison_data)



      Avg Knockdowns  Median Knockdowns  Avg Significant Strike Attempts  \
3336        0.214286                0.0                       130.428571   
3772        0.000000                0.0                         0.000000   
4024        0.000000                0.0                         8.500000   
856         0.000000                0.0                       105.333333   
1208        0.000000                0.0                         0.000000   

      Median Significant Strike Attempts  Avg Significant Strikes  \
3336                               101.5                58.428571   
3772                                 0.0                 0.000000   
4024                                 8.5                 2.500000   
856                                112.0                55.000000   
1208                                 0.0                 0.000000   

      Median Siginificant Strikes  Avg Total Strike Attempts  \
3336                         51.5                 154.642857   


## Removing the Following Statistics Due to Excessive Null Values: Fighter Reach, Fighter Stance, Fighter Weight (Weight Class Will be Used Instead)
Although Fighter Date of Birth and Fighter Height also have numerous null values, I MIGHT replace the null value with the mean/median of age and height of a fighter instead.

Fighter Reach had a lot more null values than other columns so I decided to just omit that column. Fighter Stance is either orthodox or southpaw, it does not provide enough variation to calculate meaningful averages and could negatively impact predictions, so I believe it's better to just omit the column.

In [8]:
fighter_data = pd.read_csv('/kaggle/input/d/thasankakandage/ufc-dataset-2024/ufc_fighters_median.csv')
fighter_data = fighter_data.drop(columns=['fighter_weight_lbs', 'fighter_reach_cm', 'fighter_stance'])
print(fighter_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4227 entries, 0 to 4226
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   fighter_id               4227 non-null   int64  
 1   fighter_name             4227 non-null   object 
 2   fighter_dob              3457 non-null   object 
 3   fighter_height_cm        3856 non-null   float64
 4   fighter_wins             4227 non-null   int64  
 5   fighter_losses           4227 non-null   int64  
 6   fighter_draws            4227 non-null   int64  
 7   fighter_slpm             4227 non-null   float64
 8   fighter_str_acc_%        4227 non-null   float64
 9   fighter_sapm             4227 non-null   float64
 10  fighter_str_def_%        4227 non-null   float64
 11  fighter_td_avg           4227 non-null   float64
 12  fighter_td_acc_%         4227 non-null   float64
 13  fighter_td_def_%         4227 non-null   float64
 14  fighter_sub_avg         

In [9]:

fighter_data['median_tot_strike_ratio'] = np.where(fighter_data['median_tot_strike_atts'] == 0,0,
                                                   fighter_data['median_tot_strikes'] / fighter_data['median_tot_strike_atts'])

fighter_data['median_head_strike_ratio'] = np.where(fighter_data['median_head_strike_atts'] == 0,0,
                                                    fighter_data['median_head_strikes'] / fighter_data['median_head_strike_atts'])

fighter_data['median_body_strike_ratio'] = np.where(fighter_data['median_body_strike_atts'] == 0,0,
                                                    fighter_data['median_body_strikes'] / fighter_data['median_body_strike_atts'])

fighter_data['median_leg_strike_ratio'] = np.where(fighter_data['median_leg_strike_atts'] == 0,0,
                                                   fighter_data['median_leg_strikes'] / fighter_data['median_leg_strike_atts'])

fighter_data['median_dist_strike_ratio'] = np.where(fighter_data['median_dist_strike_atts'] == 0,0,
                                                    fighter_data['median_dist_strikes'] / fighter_data['median_dist_strike_atts'])

fighter_data['median_clinch_ratio'] = np.where(fighter_data['median_clinch_atts'] == 0,0,
                                               fighter_data['median_clinchs'] / fighter_data['median_clinch_atts'])

fighter_data['median_ground_ratio'] = np.where(fighter_data['median_ground_atts'] == 0,0,
                                               fighter_data['median_grounds'] / fighter_data['median_ground_atts'])

fighter_data['median_takedown_ratio'] = np.where(fighter_data['median_takedown_atts'] == 0,0,
                                                 fighter_data['median_takedowns'] / fighter_data['median_takedown_atts'])

columns_to_drop = [
    'median_sig_strike_atts', 'median_sig_strike_atts',
    'median_sig_strikes', 'median_sig_strikes',
    'median_tot_strike_atts', 'median_tot_strike_atts',
    'median_tot_strikes', 'median_tot_strikes',
    'median_head_strike_atts', 'median_head_strike_atts',
    'median_head_strikes', 'median_head_strikes',
    'median_body_strike_atts', 'median_body_strike_atts',
    'median_body_strikes', 'median_body_strikes',
    'median_leg_strike_atts', 'median_leg_strike_atts',
    'median_leg_strikes', 'median_leg_strikes',
    'median_dist_strike_atts', 'median_dist_strike_atts',
    'median_dist_strikes', 'median_dist_strikes',
    'median_clinch_atts', 'median_clinch_atts',
    'median_clinchs', 'median_clinchs',
    'median_ground_atts', 'median_ground_atts',
    'median_grounds', 'median_grounds',
    'median_takedown_atts', 'median_takedowns',
    'median_sig_strikes', 'median_sig_strike_atts'
]

fighter_data = fighter_data.drop(columns=columns_to_drop)


### Cleaning up Null Values
Although there is quite a bit of fighters with their date of birth and height missing on the UFC stats page, [only ~150](#section3) of the fight data actually have null values for one of the fighters age/height. Since I removed these rows anyways, leaving these null values for the fighters is fine, however in the future if I were to include those 150 rows of the fight data, then I need to fill up the null values in the individual fighter dataset. 

In [10]:

null_counts_per_column = fighter_data.isnull().sum()

print("Null values per column:")
print(null_counts_per_column)

# ***
fighter_data['median_ctrl_time'] = fighter_data['median_ctrl_time'].fillna(0)
print("Null values per column after replacement:")
print(fighter_data.isnull().sum())

print(f"Number of duplicate rows: {fighter_data.duplicated().sum()}")


Null values per column:
fighter_id                    0
fighter_name                  0
fighter_dob                 770
fighter_height_cm           371
fighter_wins                  0
fighter_losses                0
fighter_draws                 0
fighter_slpm                  0
fighter_str_acc_%             0
fighter_sapm                  0
fighter_str_def_%             0
fighter_td_avg                0
fighter_td_acc_%              0
fighter_td_def_%              0
fighter_sub_avg               0
fighter_url                   0
median_knockdowns             0
median_ctrl_time            127
median_total_fight_time       0
median_submissions            0
median_reversals              0
median_tot_strike_ratio       0
median_head_strike_ratio      0
median_body_strike_ratio      0
median_leg_strike_ratio       0
median_dist_strike_ratio      0
median_clinch_ratio           0
median_ground_ratio           0
median_takedown_ratio         0
dtype: int64
Null values per column after replac

## Integrating Some Fighter Career Stats

On each individual fighter’s stat page, some career stats will be integrated into the fighter’s stats for each fight during model training. 

These career stats are:

- SLpM : Significant Strikes Landed per Minute
- Str. Acc. : Significant Striking Accuracy
- SApM : Significant Strikes Absorbed per Minute
- Str. Def. : Significant Strike Defence (the % of opponents strikes that did not land)
- TD Avg. : Average Takedowns Landed per 15 minutes
- TD Acc. : Takedown Accuracy
- TD Def. : Takedown Defense (the % of opponents TD attempts that did not land)
- Sub. Avg. : Average Submissions Attempted per 15 minutes
- Record : Wins-Losses-Draws

Out of these stats, I will only use SLpM, SApM, and their W-L Records.

- Str. Acc. - [Reasoning](#section1)
- Str. Def. - [Reasoning](#section1)
- TD Avg. - Already calculated with average takedown attempts and average takedowns within the ufc_fighters.csv file
- TD Acc. - [Reasoning](#section1)
- TD Def. - [Reasoning](#section1)
- Sub.  Avg. - Already calculated with average submission attempts and average submissions within the ufc_fighters.csv file
- Draws - Draws are extremely rare in the UFC and there is barely any data for them, only going to focus on binary classification

In [11]:
fighter_data = fighter_data.drop(columns=['fighter_str_acc_%', 'fighter_str_def_%', 'fighter_td_avg', 'fighter_td_acc_%',
                                         'fighter_td_def_%', 'fighter_sub_avg', 'fighter_url'])
print(fighter_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4227 entries, 0 to 4226
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   fighter_id                4227 non-null   int64  
 1   fighter_name              4227 non-null   object 
 2   fighter_dob               3457 non-null   object 
 3   fighter_height_cm         3856 non-null   float64
 4   fighter_wins              4227 non-null   int64  
 5   fighter_losses            4227 non-null   int64  
 6   fighter_draws             4227 non-null   int64  
 7   fighter_slpm              4227 non-null   float64
 8   fighter_sapm              4227 non-null   float64
 9   median_knockdowns         4227 non-null   float64
 10  median_ctrl_time          4227 non-null   float64
 11  median_total_fight_time   4227 non-null   float64
 12  median_submissions        4227 non-null   float64
 13  median_reversals          4227 non-null   float64
 14  median_t

Appending the individual fighter stats to their row inside the train data

In [12]:

train_data['f1_slpm'] = np.nan
train_data['f2_slpm'] = np.nan
train_data['f1_sapm'] = np.nan
train_data['f2_sapm'] = np.nan
train_data['f1_wins'] = np.nan
train_data['f2_wins'] = np.nan
train_data['f1_losses'] = np.nan
train_data['f2_losses'] = np.nan



f1_stats_list = []
f2_stats_list = []


for index, row in train_data.iterrows():
    f1_id, f2_id = row['f1_id'], row['f2_id']

    f1_stats = fighter_data[fighter_data['fighter_id'] == f1_id]
    f2_stats = fighter_data[fighter_data['fighter_id'] == f2_id]
    
    if not f1_stats.empty:
        f1_slpm, f1_sapm, f1_wins, f1_losses = f1_stats[['fighter_slpm', 'fighter_sapm', 'fighter_wins', 'fighter_losses']].values.flatten()
    else:
        f1_slpm, f1_sapm = [None, None]  
    
    if not f2_stats.empty:
        f2_slpm, f2_sapm, f2_wins, f2_losses = f2_stats[['fighter_slpm', 'fighter_sapm', 'fighter_wins', 'fighter_losses']].values.flatten()
    else:
        f2_slpm, f2_sapm = [None, None] 

    f1_stats_list.append([f1_slpm, f1_sapm])
    f2_stats_list.append([f2_slpm, f2_sapm])


    train_data.at[index, 'f1_slpm'] = f1_slpm
    train_data.at[index, 'f2_slpm'] = f2_slpm
    train_data.at[index, 'f1_sapm'] = f1_sapm
    train_data.at[index, 'f2_sapm'] = f2_sapm
    train_data.at[index, 'f1_wins'] = f1_wins
    train_data.at[index, 'f2_wins'] = f2_wins
    train_data.at[index, 'f1_losses'] = f1_losses
    train_data.at[index, 'f2_losses'] = f2_losses

 
print(train_data.head())
print(train_data.isnull().sum())

   f1_id  f2_id            f1_name           f2_name  weight_class  \
0   1566    297        Jai Herbert    Rolando Bedoya           4.0   
1   2629   2454  Azamat Murzakanov  Alonzo Menifield           7.0   
2   2015   3999  Guram Kutateladze    Jordan Vucenic           4.0   
3    107    439       Joel Alvarez      Elves Brener           4.0   
5   1125    649      Tony Ferguson    Michael Chiesa           5.0   

   f1_age_during  f2_age_during  f1_height_cm  f2_height_cm  f1_knockdowns  \
0           36.0           27.0        185.42        180.34              1   
1           35.0           36.0        177.80        182.88              1   
2           32.0           28.0        180.34        177.80              0   
3           31.0           26.0        190.50        177.80              1   
5           40.0           36.0        180.34        185.42              0   

   ...                                         fights_url  \
0  ...  http://www.ufcstats.com/fight-details/d03

Moving these columns to the end to look neater.

In [13]:

columns_to_move = ['f1_total_fight_time', 'f2_total_fight_time', 'result', 'fights_url', 'event_url']


all_columns = train_data.columns.tolist()

columns_first = [col for col in all_columns if col not in columns_to_move]

new_column_order = columns_first + columns_to_move

train_data = train_data[new_column_order]

print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 6621 entries, 0 to 7704
Data columns (total 66 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   f1_id                6621 non-null   int64  
 1   f2_id                6621 non-null   int64  
 2   f1_name              6621 non-null   object 
 3   f2_name              6621 non-null   object 
 4   weight_class         6621 non-null   float64
 5   f1_age_during        6621 non-null   float64
 6   f2_age_during        6621 non-null   float64
 7   f1_height_cm         6621 non-null   float64
 8   f2_height_cm         6621 non-null   float64
 9   f1_knockdowns        6621 non-null   int64  
 10  f2_knockdowns        6621 non-null   int64  
 11  f1_sig_strike_atts   6621 non-null   int64  
 12  f2_sig_strike_atts   6621 non-null   int64  
 13  f1_sig_strikes       6621 non-null   int64  
 14  f2_sig_strikes       6621 non-null   int64  
 15  f1_tot_strike_atts   6621 non-null   int64 

# Feature Selection
I will undergo the feature selection process to make sure I identify the most relevant features to improve the models performance.



## Features with High Multicollinearity


When features are highly correlated, it becomes difficult to isolate the individual effect of each variable on the target variable and the relationship between features and the target will be difficult to interpret. This leads to challenges in understanding which variables are mostly responsible for the outcome. Models with high multicollinearity also lead to high variance which means that the model is highly sensitive to the training data and can produce large changes in predictions with small changes in the dataset, increasing the likelihood of overfitting as the model cannot capture a more general pattern and is instead too tailored for the training data.



Note the error message below, 
"divide by zero encountered in scalar divide, vif = 1. / (1. - r_squared_i)"

This means that R^2 is equal to 1, so the feature being evaluated is perfectly predicted by other features (perfect multicollinearity). This makes it impossible to isolate the variance of a single feature, as its variation is completely explained by the other features. Perfect multicollinearity can cause instability in the model, making it difficult to determine the individual effect of correlated features.

Generally:

- VIF > 10: High multicollinearity
- 5 < VIF < 10: Intermediate multicollinearity
- VIF < 1: Little to no multicollinearity


In [14]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE

y = train_data['result']

X = train_data.drop(columns=['f1_id', 'f2_id', 'f1_name', 'f2_name', 'fights_url', 'event_url', 'result'])


def calculate_vif(dataframe):
    vif_data = pd.DataFrame()
    vif_data['feature'] = dataframe.columns
    vif_data['VIF'] = [variance_inflation_factor(dataframe.values, i) for i in range(dataframe.shape[1])]
    return vif_data


vif_data = calculate_vif(X)
print(vif_data)

/opt/conda/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                feature          VIF
0          weight_class     6.846474
1         f1_age_during    63.133326
2         f2_age_during    62.058573
3          f1_height_cm   876.494512
4          f2_height_cm   872.290337
5         f1_knockdowns     1.380156
6         f2_knockdowns     1.478519
7    f1_sig_strike_atts          inf
8    f2_sig_strike_atts          inf
9        f1_sig_strikes          inf
10       f2_sig_strikes          inf
11   f1_tot_strike_atts  1611.377078
12   f2_tot_strike_atts  1510.960557
13       f1_tot_strikes   599.577272
14       f2_tot_strikes   566.810711
15     f1_takedown_atts     4.711625
16     f2_takedown_atts     4.750098
17         f1_takedowns     4.740496
18         f2_takedowns     4.658037
19       f1_submissions     1.482987
20       f2_submissions     1.472215
21         f1_reversals     1.451662
22         f2_reversals     1.405772
23         f1_ctrl_time    10.381256
24         f2_ctrl_time     9.358808
25  f1_head_strike_atts          inf
2

<a id="section1"></a>
## Ratios and 'Out of' Stats: Simplifying UFC Models and Reducing Overfitting

Using averages to evaluate performance specifically in this UFC dataset can be problematic. For example, if a fighter has a 0.2 average for significant strikes, it indicates that 20% of their strikes are significant. However, this average can be very misleading. A fighter who has achieved 1 significant strike out of 5 attempts and another who has achieved 10 significant strikes out of 50 both have the same average of 0.2.

The volume of attempts is critical in a fight. Averages alone do not account for the number of attempts, which can greatly influence the performance of fighters. Using normalized averages alone can oversimplify the data and fail to capture the true effectiveness or consistency of a fighter's performance.

However, since our training data is almost at 60 features, and their VIF scores for 'Out of' features are extremely high, it would be better to use ratios and normalized the averages to make the model less complex and lead to less possiblities of overfitting. 




We will be outright dropping sig_strikes, as the cumulative of head, body, and leg strikes add up to the sig_strikes and sig_strike_atts

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


X['f1_tot_strike_ratio'] = np.where(X['f1_tot_strike_atts'] == 0,0,
                                    X['f1_tot_strikes'] / X['f1_tot_strike_atts'])
X['f2_tot_strike_ratio'] = np.where(X['f2_tot_strike_atts'] == 0,0,
                                    X['f2_tot_strikes'] / X['f2_tot_strike_atts'])


X['f1_head_strike_ratio'] = np.where(X['f1_head_strike_atts'] == 0,0,
                                     X['f1_head_strikes'] / X['f1_head_strike_atts'])
X['f2_head_strike_ratio'] = np.where(X['f2_head_strike_atts'] == 0,0,
                                     X['f2_head_strikes'] / X['f2_head_strike_atts'])


X['f1_body_strike_ratio'] = np.where(X['f1_body_strike_atts'] == 0,0,
                                     X['f1_body_strikes'] / X['f1_body_strike_atts'])
X['f2_body_strike_ratio'] = np.where(X['f2_body_strike_atts'] == 0,0,
                                     X['f2_body_strikes'] / X['f2_body_strike_atts'])


X['f1_leg_strike_ratio'] = np.where(X['f1_leg_strike_atts'] == 0,0,
                                    X['f1_leg_strikes'] / X['f1_leg_strike_atts'])
X['f2_leg_strike_ratio'] = np.where(X['f2_leg_strike_atts'] == 0,0,
                                    X['f2_leg_strikes'] / X['f2_leg_strike_atts'])


X['f1_dist_strike_ratio'] = np.where(X['f1_dist_strike_atts'] == 0,0,
                                     X['f1_dist_strikes'] / X['f1_dist_strike_atts'])
X['f2_dist_strike_ratio'] = np.where(X['f2_dist_strike_atts'] == 0,0,
                                     X['f2_dist_strikes'] / X['f2_dist_strike_atts'])


X['f1_clinch_ratio'] = np.where(X['f1_clinch_atts'] == 0,0,
                                X['f1_clinchs'] / X['f1_clinch_atts'])

X['f2_clinch_ratio'] = np.where(X['f2_clinch_atts'] == 0,0,
                                X['f2_clinchs'] / X['f2_clinch_atts'])


X['f1_ground_ratio'] = np.where(X['f1_ground_atts'] == 0,0,
                                X['f1_grounds'] / X['f1_ground_atts'])
X['f2_ground_ratio'] = np.where(X['f2_ground_atts'] == 0,0,
                                X['f2_grounds'] / X['f2_ground_atts'])


X['f1_takedown_ratio'] = np.where(X['f1_takedown_atts'] == 0,0,
                                  X['f1_takedowns'] / X['f1_takedown_atts'])
X['f2_takedown_ratio'] = np.where(X['f2_takedown_atts'] == 0,0,
                                  X['f2_takedowns'] / X['f2_takedown_atts'])
                                                        
columns_to_drop = [
    'f1_sig_strike_atts', 'f2_sig_strike_atts',
    'f1_sig_strikes', 'f2_sig_strikes',
    'f1_tot_strike_atts', 'f2_tot_strike_atts',
    'f1_tot_strikes', 'f2_tot_strikes',
    'f1_head_strike_atts', 'f2_head_strike_atts',
    'f1_head_strikes', 'f2_head_strikes',
    'f1_body_strike_atts', 'f2_body_strike_atts',
    'f1_body_strikes', 'f2_body_strikes',
    'f1_leg_strike_atts', 'f2_leg_strike_atts',
    'f1_leg_strikes', 'f2_leg_strikes',
    'f1_dist_strike_atts', 'f2_dist_strike_atts',
    'f1_dist_strikes', 'f2_dist_strikes',
    'f1_clinch_atts', 'f2_clinch_atts',
    'f1_clinchs', 'f2_clinchs',
    'f1_ground_atts', 'f2_ground_atts',
    'f1_grounds', 'f2_grounds',
    'f1_takedowns', 'f2_takedowns',
    'f1_takedown_atts', 'f2_takedown_atts',
    'f1_sig_strikes', 'f2_sig_strikes',
    'f2_sig_strike_atts', 'f2_sig_strike_atts'
]

X = X.drop(columns=columns_to_drop)


print(X.info())
print(X.head())


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


vif_data = calculate_vif(X)
print(vif_data)

<class 'pandas.core.frame.DataFrame'>
Index: 6621 entries, 0 to 7704
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   weight_class          6621 non-null   float64
 1   f1_age_during         6621 non-null   float64
 2   f2_age_during         6621 non-null   float64
 3   f1_height_cm          6621 non-null   float64
 4   f2_height_cm          6621 non-null   float64
 5   f1_knockdowns         6621 non-null   int64  
 6   f2_knockdowns         6621 non-null   int64  
 7   f1_submissions        6621 non-null   int64  
 8   f2_submissions        6621 non-null   int64  
 9   f1_reversals          6621 non-null   int64  
 10  f2_reversals          6621 non-null   int64  
 11  f1_ctrl_time          6621 non-null   float64
 12  f2_ctrl_time          6621 non-null   float64
 13  f1_slpm               6621 non-null   float64
 14  f2_slpm               6621 non-null   float64
 15  f1_sapm               6621

/opt/conda/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                 feature         VIF
0           weight_class    6.361408
1          f1_age_during   62.151171
2          f2_age_during   61.145198
3           f1_height_cm  856.146199
4           f2_height_cm  852.200352
5          f1_knockdowns    1.481708
6          f2_knockdowns    1.570520
7         f1_submissions    1.483986
8         f2_submissions    1.455394
9           f1_reversals    1.407755
10          f2_reversals    1.366974
11          f1_ctrl_time    3.991608
12          f2_ctrl_time    3.794556
13               f1_slpm   12.758632
14               f2_slpm   11.826339
15               f1_sapm   12.982544
16               f2_sapm    9.832287
17               f1_wins   11.345471
18               f2_wins   10.839285
19             f1_losses    8.667856
20             f2_losses    8.020116
21   f1_total_fight_time         inf
22   f2_total_fight_time         inf
23   f1_tot_strike_ratio   31.970925
24   f2_tot_strike_ratio   29.514298
25  f1_head_strike_ratio   13.576415
2

<a id="section2"></a>
## Testing Linear Seperability
We can test linear seperability by using SVM and overfit on the training data.
1. Start with a large C. 
2. Train the model.
3. Test on the training set.
4. If we get 100% accuracy, the data is linearly seperable.
A large C penalizes incorrect data points heavily, so it will make the optimizer have 0 error in classification in order to minimize the loss function. Doing this overfits the data.

Accuracy of 89% so not linearly seperable

In [16]:

# from sklearn.svm import SVC

# svm_model = SVC(C=2^32, kernel="linear")  
# svm_model.fit(X_train, y_train)


# y_pred = svm_model.predict(X_train)

# print(f'Accuracy: {accuracy_score(y_train, y_pred):.5f}')

# Model Choice
For hyperparameter tuning, I used Grid Search and Bayesian Optimization.

## SVM
- **C**: Controls the penalty for misclassified data points. A small value lets the penalty to be small and the model to be more tolerant of misclassified points. This means that the model is more flexible so it can generalize more, but can also underfit. A large value penalizes incorrect data points heavily, fitting the training data as accurately as possible leading to less tolerance of errors and leading to possible overfitting and less generalizaiton. 
2. **kernel**: Function to transform the data into a higher-dimensional space where it can linearly seperated. Lets the SVM find a hyperplane that can separate classes that are not linearly separable in the original input space.

3. **gamma**: how far the influence of a single training example reaches for the rbf kernel. A high value limits the influence of a single training, results in the model closely fitting the training data, capturing more details and patterns but leading to possible overfitting. A low value allows greater influence of a single training example, making it less sensitive to individual training examples allowing the model to generalize better, but leading to potential underfitting. 

The data is not linearly seperable [as seen above](#section2). We can still use the "rbf" kernal to create complex boundaries to seperate non-linear data by transforming the input space into a higher dimensional space.

In [17]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# # parameter grid for Grid Search and Random Search
# param_grid = {
#     'C': [0.1, 1, 10, 100, 1000], 
#     'gamma': [0.001, 0.01, 0.1, 1, 10, 100],  
#     'kernel': ['rbf'] 
# }
# # # Grid Search
# parameter_search = GridSearchCV(estimator=SVC(probability=True), param_grid=param_grid, 
#                            scoring='accuracy', cv=5, verbose=0)


# parameter_search.fit(X_train, y_train)


# best_params = parameter_search.best_params_
# best_model = parameter_search.best_estimator_

# y_pred = best_model.predict(X_test)

# print(f'Best parameters: {best_params}')
# print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')
# # Best parameters: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
# # Accuracy: 0.89







# svm_model = SVC(C=100, kernel='rbf', gamma=0.001, probability = True)  
# svm_model.fit(X_train, y_train)


# y_pred = svm_model.predict(X_test)

# print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')

<a id="section4"></a>
## Random Forest Classifier
- **n_estimators** (number of trees): Number of decision trees that will be created in the forest. More trees will lead to a more accurate model, however it will also be more computationally expensive and the growth in accuracy will eventually plateau.

- **max_depth**: Maximum depth of the decision trees from root to leaf to help control complexity. A higher depth will lead to a more accurate model, but it can also lead to the tree capturing noise and small changes leading to overfitting the data, and of course, more computing power is needed for deeper trees. A lower depth will lead to decisions based on less splits and will be less complex, so less likely to overfit and instead become more generalized, but can also lead to underfitting.  
- **min_samples_split**: Minimum number of samples required to split a node in a decision tree. A higher number of samples will help stop the creation of nodes that are too specific, which can lead to overfitting. Also, a high value means that nodes require a large number of samples to make a split which results in fewer splits and therefore a low depth tree. A low number of samples lets nodes split even with few samples, creating deeper trees.
- **min_samples_leaf**: Minimum number of samples required to be in a leaf node. A high number of samples lead to low trees because they will have to have a lot of samples in each leaf, preventing the trees to capture patterns and leading to underfitting. A low number of samples will lead to trees growing deeper with smaller leaf nodes, capturing more detailed patterns but also potential overfitting. Integer values means # of samples, while float values means % of samples.
- **max_features**: Number of features that are considered when splitting a node in a decision tree. Limiting the number of features creates diversity with the trees in the forest which helps improve the generalization and reduces overfitting. High values will result in each tree having access to most of the features, leading to accurate trees but might reduce the generalization and leads to overfitting. Low values results in each tree seeing a small set of features, increasing generalization  but less accurate trees and potential underfitting. Integer values means # of features, while float values mean % of features


Random Forests are less affected by multicollinearity as it randomly select a subset of features for each tree split. This random selection reduces the impact of multicollinearity because each tree is not affected by all features simultaneously. However as seen above, many of the features were highly correlated with each other so a lot of the features selected will affect the model performance. In the presence of multicollinearity, feature importance scores might not clearly reflect the true importance of features because correlated features may share importance. Random Forests might assign importance to multiple correlated features rather than isolating the contribution of each.

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Integer


# parameter grid for Grid Search and Random Search
# parameter_grid = {
#     'n_estimators': [50, 100, 150],        
#     'max_depth': [None, 10, 20, 30],       
#     'min_samples_split': [2, 5, 10],    
#     'min_samples_leaf': [1, 2, 4],    
#     'max_features': ['sqrt', 'log2']  
# }


# # Grid Search
# parameter_search = GridSearchCV(estimator=RandomForestClassifier(), 
#                            param_grid=parameter_grid, 
#                            cv=5,                            
#                            verbose=0)                      
# # Best Parameters: {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 150}

# # Random Search
# parameter_search = RandomizedSearchCV(estimator=RandomForestClassifier(),
#     param_distributions=parameter_grid,
#     n_iter=1000,
#     cv=5,
#     verbose=0
# )
# # Best Parameters: {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 150}


# # parameter space for Bayesian Optimization
# parameter_space = {
#     'n_estimators': Integer(10, 200),
#     'max_depth': Integer(1, 30),
#     'min_samples_split': Integer(2, 20),
#     'min_samples_leaf': Integer(1, 20),
#     'max_features': ['sqrt', 'log2']
# }


# # Bayesian Optimization
# parameter_search = BayesSearchCV(
#     estimator=RandomForestClassifier(),
#     search_spaces=parameter_space,
#     n_iter=500,  
#     cv=5,      
#     verbose=0
# )
# # Best Parameters: {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 150}


# parameter_search.fit(X_train, y_train)

# best_params = grid_search.best_params_
# best_rf_model = grid_search.best_estimator_

# y_pred = best_rf_model.predict(X_test)

# print(f'Best Parameters: {best_params}')
# print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')










# more general params?
# n_estimators=50,max_depth=10,min_samples_split=20,min_samples_leaf=1,max_features='sqrt', random_state=42)


# rf_model = RandomForestClassifier(n_estimators=50,max_depth=10,min_samples_split=20,min_samples_leaf=1,max_features='sqrt', random_state=42)
# rf_model.fit(X_train, y_train)


# y_pred = rf_model.predict(X_test)



# print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')

## XGBoost
- **n_estimators**: [Definition](#section4)
- **max_depth**: [Definition](#section4)

- **subsample**: Controls the fraction of randomly sampled training ddata that is used for each tree. A high value close to 1 means that most of the training data is used for each tree, which generally leads to less variance and potentially more overfitting. A low value introduces randomness and can help in preventing overfitting by making the model less dependent on any particular subset of the data.  Lower values can improve generalization as each tree is trained on a slightly different subset of the data, but if the value is too low then model may not have enough data to train each tree effectively, leading to underfitting.
- **colsample_bytree**: Controls the fraction of randomly sampled features used for each tree. A high value means that most features are used for each tree which leads to less feature randomness but can lead to overfitting. A smaller value results in smaller and less complex models, which can help prevent overfitting but has risks of underfitting.

- **min_child_weight**: Controls the the number of samples required to create a new child node in a decision tree. When making a tree, the nodes only split if it would result in child nodes with the sum of weights being more than the min_child_weight. A larger value means that more instances are required in a node before it can be split which produces simpler trees with fewer splits, this can prevent the tree from capturing important patterns in the data, leading to underfitting. Smaller values means that fewer instances are required to create child nodes, allowing the model to create more splits and leads to complex trees that capture intricate patterns in the training data, but this increase the risk of overfitting.

Using the DART booster to help prevent overfitting.

In [19]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from skopt import BayesSearchCV

# parameter grid for Grid Search and Random Search
# param_grid = {
#     'n_estimators': [50, 100, 200],         
#     'learning_rate': [0.01, 0.1, 0.2],
#     'max_depth': [3, 6, 9],                    
#     'min_child_weight': [1, 3, 5],             
#     'subsample': [0.8, 1],                
#     'colsample_bytree': [0.8, 1]}

# # initialize model
# xgb_model = xgb.XGBClassifier(
#     booster='dart',        
#     objective='binary:logistic',
#     use_label_encoder=False,
#     eval_metric='logloss'
# )


# Grid Search
# parameter_search = GridSearchCV(
#     estimator=xgb_model,
#     param_grid=param_grid,
#     scoring='accuracy',     
#     cv=2,                
#     verbose=1)
# Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 200, 'subsample': 1}
# Accuracy: 0.88


# # parameter space for Bayesian Optimization
# param_space = {
#     'n_estimators': (50, 200),          
#     'learning_rate': (0.01, 0.2, 'log-uniform'),
#     'max_depth': (3, 9),                    
#     'min_child_weight': (1, 5),             
#     'subsample': (0.8, 1.0),                
#     'colsample_bytree': (0.8, 1.0)
# }



# # Bayesian Optimization
# parameter_search = BayesSearchCV(
#     estimator=xgb_model,
#     search_spaces=param_space,
#     scoring='accuracy',
#     n_iter=32,
#     cv=2,      
#     verbose=1,
# )
# # Best Parameters:[('colsample_bytree',0.9346),('learning_rate',0.097488),('max_depth',3),('min_child_weight',5), ('n_estimators',159), ('subsample',1)]
# # Accuracy: 0.88









# parameter_search.fit(X_train, y_train)

# best_params = parameter_search.best_params_
# best_model = parameter_search.best_estimator_

# y_pred = best_model.predict(X_test)

# print(f'Best Parameters: {best_params}')
# print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')
















xgb_model = xgb.XGBClassifier(booster="dart",  use_label_encoder=False, eval_metric='logloss', colsample_bytree=0.8, learning_rate=0.01, max_depth=3,
                             min_child_weight=5, n_estimators=50, subsample=0.8, objective='binary:logistic' )
xgb_model.fit(X_train, y_train)


y_pred = xgb_model.predict(X_test)


print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')


Accuracy: 0.82


# Testing

To test, simply type in two fighters names, and the weight class they will be fighting in.

In [20]:
from datetime import datetime
# DON'T USE THESE NAMES! These names are shared with atleast 1 other fighter
# An age input will later be developed incase a duplicate name is chosen

# Mike Davis 
# Joey Gomez 
# Tony Johnson    
# Michael McDonald  
# Jean Silva    
# Bruno Silva

# these are your three inputs
#################################
fighter1_name = "Dricus Du Plessis"
fighter2_name = "Israel Adesanya"
weight_class = "Middleweight"
#################################

df = pd.DataFrame({
    'weight_class': [weight_class]
})
df['weight_class_value'] = df.apply(map_weight_class, axis=1)
weight_class_value = df['weight_class_value'].iloc[0]
print(f'Weight class value for "{weight_class}": {weight_class_value}')

today = datetime.now()
if 'fighter_dob' in fighter_data.columns:
    fighter_data['fighter_dob'] = fighter_data['fighter_dob'].apply(
        lambda dob: today.year - pd.to_datetime(dob).year - (
            (today.month, today.day) < (pd.to_datetime(dob).month, pd.to_datetime(dob).day)
        )
    )


    fighter_data = fighter_data.rename(columns={'fighter_dob': 'fighter_age_during'})
print(fighter_data.info())
fighter1_data = fighter_data[fighter_data["fighter_name"] == fighter1_name]
fighter2_data = fighter_data[fighter_data["fighter_name"] == fighter2_name]
training_columns = X.columns.tolist()
print(X.info())

non_average_stats = ['fighter_wins', 'fighter_losses', 'fighter_slpm', 'fighter_sapm', 'fighter_age_during', 'fighter_height_cm']
features = []

for column in training_columns:

    if column.startswith('f1_'): 
        if column.replace("f1_", "fighter_") not in non_average_stats:
            feature = column.replace("f1_", "median_")
            if feature not in features:
                features.append(feature)
        else:
            feature = column.replace("f1_", "fighter_")
            if feature not in features:
                features.append(feature)
    elif column.startswith('f2_'): 
        if column.replace("f2_", "fighter_") not in non_average_stats:
            feature = column.replace("f2_", "median_")
            if feature not in features:
                features.append(feature)
            
        else:
            feature = column.replace("f2_", "fighter_")
  
        

fighter1_features = fighter1_data[features].columns.tolist()
fighter2_features = fighter2_data[features].columns.tolist()


fighter1_data_list = [(f'f1_{col_name}', fighter1_data[col_name].values[0]) for col_name in fighter1_features]
fighter2_data_list = [(f'f2_{col_name}', fighter2_data[col_name].values[0]) for col_name in fighter2_features]


interleaved_feature_values_list = [val for pair in zip(fighter1_data_list, fighter2_data_list) for val in pair]


for col_name, value in interleaved_feature_values_list:
    print(f"{col_name}: {value}")


fighter1_data_filtered = fighter1_data[features]
fighter2_data_filtered = fighter2_data[features]

    
fighter1_array = fighter1_data_filtered.values.flatten()
fighter2_array = fighter2_data_filtered.values.flatten()

interleaved_data = np.empty(len(X.columns.tolist()))

interleaved_data[0] = weight_class_value


interleaved_data[1::2] = fighter1_array
interleaved_data[2::2] = fighter2_array
interleaved_data = interleaved_data.reshape(1, -1)

single_input = scaler.transform(interleaved_data)


                                
    
    
    
    
# # SVM Model
# prediction_probs = svm_model.predict_proba(single_input)
# single_pred = svm_model.predict(single_input)
# prediction_probability = prediction_probs[0][single_pred[0]]

# # Random Forest Classifier Model - best so far
# prediction_probs = rf_model.predict_proba(single_input)  
# print(prediction_probs)
# single_pred = rf_model.predict(single_input)  
# prediction_probability = prediction_probs[0][single_pred[0]]  

# XGBoost Model
prediction_probs = xgb_model.predict_proba(single_input)  
print(prediction_probs)
single_pred = xgb_model.predict(single_input)  
prediction_probability = prediction_probs[0][single_pred[0]]  



 
    
if single_pred[0] == 0:
    predicted_winner = fighter1_name
elif single_pred[0] == 1:
    predicted_winner = fighter2_name
else:
    print(single_prediction[0])
    predicted_winner = "Unknown"

print(f'Predicted Winner: {predicted_winner}')
print(f'Probability: {prediction_probability:.2f}')


Weight class value for "Middleweight": 6
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4227 entries, 0 to 4226
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   fighter_id                4227 non-null   int64  
 1   fighter_name              4227 non-null   object 
 2   fighter_age_during        3457 non-null   float64
 3   fighter_height_cm         3856 non-null   float64
 4   fighter_wins              4227 non-null   int64  
 5   fighter_losses            4227 non-null   int64  
 6   fighter_draws             4227 non-null   int64  
 7   fighter_slpm              4227 non-null   float64
 8   fighter_sapm              4227 non-null   float64
 9   median_knockdowns         4227 non-null   float64
 10  median_ctrl_time          4227 non-null   float64
 11  median_total_fight_time   4227 non-null   float64
 12  median_submissions        4227 non-null   float64
 13  median_reversals      

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


# To Add
- Further add data preprocessing, e.g., checking if certain rows have too many 0's, adding age input if user inputs name belonging to multiple fighters
- Try NNs

- Further data analysis